In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.8.2 on the current Colab Runtime...
2023-05-20 15:47:08 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.8/julia-1.8.2-linux-x86_64.tar.gz [135859273/135859273] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.8

Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then
jump to the 'Checking the Installation' section.


In [ ]:
versioninfo()

Julia Version 1.8.2
Commit 36034abf260 (2022-09-29 15:21 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, broadwell)
  Threads: 2 on 2 virtual cores
Environment:
  LD_LIBRARY_PATH = /usr/local/nvidia/lib:/usr/local/nvidia/lib64
  JULIA_NUM_THREADS = 2


Add new code cells by clicking the `+ Code` button (or _Insert_ > _Code cell_).

Have fun!

<img src="https://raw.githubusercontent.com/JuliaLang/julia-logo-graphics/master/images/julia-logo-mask.png" height="100" />

In [ ]:
using Pkg

In [ ]:
Pkg.add(url="https://github.com/JuliaQuantumControl/GRAPELinesearchAnalysis.jl.git")

     Cloning git-repo `https://github.com/JuliaQuantumControl/GRAPELinesearchAnalysis.jl.git`
    Updating git-repo `https://github.com/JuliaQuantumControl/GRAPELinesearchAnalysis.jl.git`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed Calculus ───────────────────────── v0.5.1
   Installed libfdk_aac_jll ─────────────────── v2.0.2+0
   Installed JpegTurbo_jll ──────────────────── v2.1.91+0
   Installed x265_jll ───────────────────────── v3.5.0+0
   Installed Libmount_jll ───────────────────── v2.35.0+0
   Installed GR_jll ─────────────────────────── v0.72.5+0
   Installed OffsetArrays ───────────────────── v1.12.9
   Installed HypergeometricFunctions ────────── v0.3.16
   Installed LERC_jll ───────────────────────── v3.0.0+1
   Installed StatsFuns ──────────────────────── v1.3.0
   Installed NonlinearSolve ─────────────────── v1.6.0
   Installed LoggingExtras ──────────────────── v1.0.0
   Installed Opus_jll ────────────────────

In [ ]:
Pkg.add("QuantumControl")

   Resolving package versions...
   Installed QuantumControl ─ v0.8.0
   Installed Krotov ───────── v0.5.3
    Updating `~/.julia/environments/v1.8/Project.toml`
  [8a270532] + QuantumControl v0.8.0
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [b05dcdc7] + Krotov v0.5.3
  [8a270532] + QuantumControl v0.8.0
Precompiling project...
  ✓ Krotov
  ✓ QuantumControl
  2 dependencies successfully precompiled in 15 seconds. 270 already precompiled.


In [ ]:
using GRAPELinesearchAnalysis

In [ ]:
using QuantumControl

In [ ]:
Pkg.add("QuantumControlBase")

   Resolving package versions...
    Updating `~/.julia/environments/v1.8/Project.toml`
  [f10a33bc] + QuantumControlBase v0.8.3
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [ ]:
using QuantumControlBase

In [ ]:
ϵ1(t) = 0.2 * QuantumControl.Shapes.flattop(t, T=300, t_rise=0.3, func=:blackman);
ϵ2(t) = 0.2 * QuantumControl.Shapes.flattop(t, T=300, t_rise=0.3, func=:blackman);
ϵ3(t) = 0.2 * QuantumControl.Shapes.flattop(t, T=300, t_rise=0.3, func=:blackman);
ϵ4(t) = 0.2 * QuantumControl.Shapes.flattop(t, T=300, t_rise=0.3, func=:blackman);

In [ ]:
"""Two-level-system Hamiltonian."""
function hamiltonian(Ω1=1.0, ϵ1=ϵ1,ϵ2=ϵ2,ϵ3=ϵ3,ϵ4=ϵ4, Ω2=1.0, J=0.01)
    Iσ̂_z = ComplexF64[
        1  0  0  0
        0 -1  0  0
        0  0  1  0
        0  0  0  -1
    ]
    Iσ̂_x = ComplexF64[
        0  1  0  0
        1  0  0  0
        0  0  0  1
        0  0  1  0
    ]
    σ̂_xI = ComplexF64[
        0  0  1  0
        0  0  0  1
        1  0  0  0
        0  1  0  0
    ]
    Iσ̂_y = ComplexF64[
        0  -1im  0  0
        1im  0  0  0
        0  0  0  -1im
        0  0  1im  0
    ]
    σ̂_yI = ComplexF64[
        0  0  -1im  0
        0  0  0  -1im
        1im  0  0  0
        0  1im  0  0
    ]
    σ̂_zI = ComplexF64[
        1  0  0  0
        0  1  0  0
        0  0 -1  0
        0  0  0  -1
    ]
    σ̂_x_x = ComplexF64[
        0  0  0  1
        0  0  1  0
        0  1  0  0
        1  0  0  0
    ]
    σ̂_y_y = ComplexF64[
        0  0  0 -1
        0  0  1  0
        0  1  0  0
       -1  0  0  0
    ]
    σ̂_z_z = ComplexF64[
        1  0  0  0
        0 -1  0  0
        0  0 -1  0
        0  0  0  1
    ]
    Ĥ₀ = -0.5 * (Ω1 * Iσ̂_z + Ω2 * σ̂_zI) + J * (σ̂_z_z)
    Ĥ₁ = Iσ̂_x
    Ĥ2 = σ̂_xI
    Ĥ3 = Iσ̂_y
    Ĥ4 = σ̂_yI
    return (Ĥ₀, (Ĥ₁, ϵ1), (Ĥ2, ϵ2), (Ĥ3, ϵ3), (Ĥ4, ϵ4))
end;

In [ ]:
H = hamiltonian();

In [ ]:
tlist = collect(range(0, 300, length=3000));

In [ ]:
using LinearAlgebra

In [ ]:
function ket(label)
    result = Dict("00" => Vector{ComplexF64}([1, 0, 0, 0]), "01" => Vector{ComplexF64}([0, 1, 0, 0]), "10" => Vector{ComplexF64}([0, 0, 1, 0]), "11" => Vector{ComplexF64}([0, 0, 0, 1]))
    return result[string(label)]
end;

In [ ]:
basis = [ket("00"),ket("01"),ket("10"),ket("11")];

In [ ]:
basis_tgt=Array[]
for i in 1:100
   b= transpose(RandU[i]) * basis
   push!(basis_tgt, b)

end


In [ ]:
objectives=Array[]
for i in 1:100
   obj = [Objective(initial_state = Ψ, target_state = Ψtgt, generator = H) for
    (Ψ, Ψtgt) ∈ zip(basis, basis_tgt[i])];
    push!(objectives, obj)
end

In [ ]:
problem=[]
for i in 1:100

    prob = ControlProblem(
    objectives=objectives[i],
    tlist=tlist,
    pulse_options=Dict(),
    iter_stop=500,
    J_T=QuantumControl.Functionals.J_T_sm,
    check_convergence=res -> begin
        ((res.J_T < 1e-3) && (res.converged = true) && (res.message = "J_T < 10⁻³"))
    end,
    );
    push!(problem, prob)
end

In [ ]:
Pkg.add("QuantumPropagators")
using QuantumPropagators

   Resolving package versions...
    Updating `~/.julia/environments/v1.8/Project.toml`
  [7bf12567] + QuantumPropagators v0.6.0
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [ ]:
function store_pulses(wrk, iteration, _...)
    L = length(wrk.controls)
    ϵ_opt = reshape(wrk.pulsevals, L, :)
    opt_pulses = [QuantumControl.Controls.discretize_on_midpoints(ϵ_opt[l, :], tlist) for l=1:L]
    return Tuple(opt_pulses)
end

store_pulses (generic function with 1 method)

In [ ]:
Pkg.add("GRAPE")
using GRAPE

   Resolving package versions...
    Updating `~/.julia/environments/v1.8/Project.toml`
  [6b52fcaf] + GRAPE v0.5.4
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [ ]:
wrk=[]
for i in 1:100
  wrkk=GRAPE.GrapeWrk(problem[i]; )
  push!(wrk, wrkk)
end


In [ ]:
results=[]

for i in 1:100
  opt_result_LBFGSB = optimize(

      problem[i],
      method = :grape,
      force = true,
      info_hook = QuantumControlBase.chain_infohooks(

          QuantumControl.GRAPE.print_table,
          store_pulses
      )
  );
  push!(results, opt_result_LBFGSB)
end

#opt_result_LBFGSB

 iter.        J_T     |∇J_T|       ΔJ_T   FG(F)    secs
     0   7.92e-01   8.12e-01        n/a    1(0)    10.1
     1   5.83e-01   2.63e+00  -2.09e-01    2(0)     2.2
     2   5.39e-01   3.08e+00  -4.37e-02    3(0)     2.2
     3   3.72e-01   3.01e+00  -1.67e-01    1(0)     1.4
     4   1.72e-01   1.61e+00  -2.00e-01    2(0)     1.7
     5   8.85e-02   7.57e-01  -8.34e-02    1(0)     0.9
     6   6.49e-02   6.01e-01  -2.37e-02    1(0)     0.7
     7   2.35e-02   5.54e-01  -4.14e-02    1(0)     0.8
     8   1.59e-02   3.08e-01  -7.65e-03    2(0)     1.5
     9   1.28e-02   1.87e-01  -3.06e-03    1(0)     0.8
    10   9.69e-03   1.98e-01  -3.12e-03    1(0)     0.7
    11   6.06e-03   2.30e-01  -3.63e-03    1(0)     0.8
    12   2.91e-03   2.76e-01  -3.16e-03    1(0)     0.8
    13   9.68e-04   1.36e-01  -1.94e-03    1(0)     0.8
 iter.        J_T     |∇J_T|       ΔJ_T   FG(F)    secs
     0   9.08e-01   1.39e+00        n/a    1(0)     0.7
     1   2.40e-01   1.83e+00  -6.68e-01    3(0) 

In [ ]:
t_conv_4G=[]

for i in 1:25
  t_sec = 0
  t_conv_i = []
   for j = 1: length(iter_sec_2Q_4G[i])
      t_sec = iter_sec_2Q_4G[i][j] + t_sec
      push!(t_conv_i, t_sec)
  end
  push!(t_conv_4G, t_conv_i)
end
println(t_conv_4G)

Any[Any[10.1, 12.3, 14.5, 15.9, 17.6, 18.5, 19.2, 20.0, 21.5, 22.3, 23.0, 23.8, 24.6, 25.400000000000002], Any[0.7, 3.8, 5.3, 6.1, 6.8999999999999995, 7.699999999999999, 8.399999999999999, 9.2, 10.0, 10.7, 11.5, 12.3, 14.8, 15.600000000000001, 16.5], Any[0.8, 1.6, 5.5, 7.1, 7.8999999999999995, 9.2, 10.6, 12.0, 12.8, 13.5, 14.3, 15.0], Any[0.8, 2.3, 3.0999999999999996, 4.6, 5.3, 6.1, 6.8999999999999995, 8.299999999999999, 9.2, 9.899999999999999, 10.7, 11.5], Any[0.8, 2.2, 3.0, 4.5, 5.3, 6.1, 8.2, 9.299999999999999, 10.1, 10.9, 11.6, 12.4, 13.200000000000001, 13.9, 14.700000000000001, 15.4], Any[0.8, 3.0999999999999996, 5.5, 6.9, 7.7, 8.4, 9.200000000000001, 9.9, 10.700000000000001], Any[0.8, 1.5, 3.0, 4.6, 6.0, 7.0, 7.7, 8.5, 9.2, 9.899999999999999, 10.7, 11.5, 12.3, 13.0, 13.8, 14.600000000000001, 15.3, 16.1, 17.3, 18.5], Any[0.8, 2.3, 3.0, 5.9, 6.7, 7.5, 11.2, 12.2, 13.0, 13.8, 14.5, 15.3, 16.1, 16.900000000000002, 17.6, 19.1, 19.900000000000002, 20.700000000000003, 21.500000000000004

In [ ]:
N=25
t_conv_avg_4G=0
sum=0
for i in 1:N

  sum=t_conv_4G[i][end] + sum

end

t_conv_avg_4G= sum/N
println(t_conv_avg_4G)

15.116000000000001
